In [1]:
import numpy as np
import math
from sklearn.metrics import classification_report

## Micro/Macro Avg, percision, recall from Confusion Matrix

In [11]:
cfm = np.array([[9,2,3,3,3],
                [1,9,2,4,4],
                [1,2,11,2,4],
                [4,3,1,10,2],
                [1,1,3,1,14]
                ])
tn = ['bird','car','dog','horse','ship']

l = len(cfm)
ypred = np.array([])
ytrue = np.array([])
# Creating dummy ytrue, ypred for comparison. sklearn will do the rest of the job
for i in range(l):
    for j in range(l):
        tmp = np.ones([cfm[i,j]])
        ytrue = np.append(ytrue,i*tmp, axis = 0)
        ypred = np.append(ypred,j*tmp, axis = 0)

cfr = classification_report(ytrue, ypred,target_names=tn)
print(cfr)

              precision    recall  f1-score   support

        bird       0.56      0.45      0.50        20
         car       0.53      0.45      0.49        20
         dog       0.55      0.55      0.55        20
       horse       0.50      0.50      0.50        20
        ship       0.52      0.70      0.60        20

    accuracy                           0.53       100
   macro avg       0.53      0.53      0.53       100
weighted avg       0.53      0.53      0.53       100



In [22]:
def describe(feature, cfr):
    pass

cfrd = classification_report(ytrue, ypred,target_names=tn, output_dict=True)
cfrd['bird']['recall']

0.45

## Bayes Rule

In [3]:
# Calculating P(A|B)

#P(A) - Prior
pa = 0.22
#P(B) - Prior of evidence
pb = 0.2
#P(B|A) - Posterior
pba = 0.02

pab = pba*pa/pb
print(pb)

0.2


## Entropy

In [4]:
def entropy(ps, i = ''):
    print('H'+i+' = ' + ' + '.join([f'-{p}*log2({p})' for p in ps]))
    es = [0 if not p else -p*math.log2(p) for p in ps]
    entr = np.sum(es)
    print('= ' + ' + '.join([f'{e:.4g}' for e in es]) + f' = {entr:.4g}')
    return entr

In [7]:
ps = [0.6,0.4]
entropy(ps)

H = -0.6*log2(0.6) + -0.4*log2(0.4)
= 0.4422 + 0.5288 = 0.971


0.9709505944546686

## Normal/Gaussian Distribution

In [6]:
"""
Calculates the probability that event X occurs based on probability density function
"""
pdf = lambda mu, sigma, x: math.exp(-((x-mu)/sigma)**2/2)/(sigma*2*math.pi)

pdf(1,3,4)

0.03217745087668464